In [194]:
import scipy.io
import numpy as np
from numpy import linalg as LA
import matplotlib
import time
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sympy


In [195]:
def show_img(img):
    temp = img.copy()
    temp.resize((46,56))
    im = Image.fromarray(temp.T)
    im.show()

In [196]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

D,N = raw_data.shape

raw_data = np.transpose(raw_data)

training_data = np.empty([int(520*0.8), 2576])
testing_data = np.empty([int(520*0.2), 2576])

# create training and test data
for x in range(52):
        # 8/2 ratio for training and testing datasets
	training_data[x*8:(x+1)*8] = raw_data[x*10:x*10+8]
	testing_data[x*2:(x+1)*2] = raw_data[x*10+8:(x+1)*10]

raw_data = np.transpose(raw_data)
training_data = np.transpose(training_data)
testing_data = np.transpose(testing_data)



In [197]:
# get mean image matrix
mean_face = training_data.mean(axis=1).reshape(-1,1)

show_img(mean_face)

In [198]:
A = training_data - mean_face
S = (1/N)*np.dot(np.transpose(A),A)
# S = np.cov(np.transpose(A))

w, v = LA.eig(S)

# u = principal components
u = np.dot(A,v)
u /= LA.norm(u,ord=2,axis=0)

id = np.argsort(np.abs(w))[::-1]
w = w[id]
u = np.real(u[:,id])
u_norm = u



In [199]:
# for i in range(10):
#     asdf = u_norm[:,i].reshape(46,56)
#     im = Image.fromarray(asdf)
#     im.show()

In [200]:
input = training_data
delta = input - mean_face

Compute weights for testing images

In [201]:
n_face = 416
eigface = u_norm[:,:n_face]
weights = np.dot(delta.T, eigface)
print(weights.shape)
print(weights)

(416, 416)
[[-5.44059059e+02 -6.51006537e+01 -3.12137872e+02 ... -3.02899252e+00
   6.60685437e-02 -1.22085028e+01]
 [-1.83831853e+02  3.31273057e+02 -6.54260422e+02 ...  2.37661483e-01
   4.00909943e+00 -2.07952616e+01]
 [-5.40629185e+01 -7.89074429e+02  1.60230856e+02 ... -1.15904740e+01
  -7.48810772e+00 -7.26253258e+01]
 ...
 [ 4.22721333e+02 -4.56929762e+02  2.15754140e+02 ...  1.38725200e+00
   9.56521961e-01  1.91225812e+01]
 [ 8.92658793e+02  6.81663147e+01 -5.34142508e+02 ... -6.95068027e+00
   5.04640212e+00  1.61558698e+01]
 [ 6.29509947e+02  7.01129638e+01 -5.92814779e+02 ...  1.26351798e+00
  -1.94788207e-03 -1.57764936e+00]]


In [202]:
reconstructed_faces = np.matmul(weights, eigface.T)

print(reconstructed_faces)

face = []
for i in range(len(reconstructed_faces)):
    face.append(reconstructed_faces[i][:] + mean_face.T)


for x in face[:20]:
    show_img(x)

[[-16.58509086 -19.32023601 -17.15359452 ... -48.04959339 -47.38508555
  -40.17461459]
 [  7.51229875   6.62529183   9.97845919 ... -45.21466052 -47.78950002
  -46.13004647]
 [ 69.10014682  62.29649546  54.77554132 ... -64.21101319 -67.23056404
  -63.86103125]
 ...
 [ 58.05955713  47.87852019  42.36457226 ... -23.44730186 -31.90947128
  -36.33723331]
 [-62.90679491 -54.14029985 -49.5898034  ... -43.50433229 -34.04919583
  -16.32183509]
 [-52.70566425 -60.25279666 -37.31708386 ...   2.15476829   6.11560058
   12.77020775]]
